In [1]:
import pandas as pd

# define constant
SENTENCES_PATH = "../data/processed/100测试语音/data.csv"
KEYWORDS_PATH = "../data/raw/Keyword list of NLP.xlsx"

# load data
sentences = pd.read_csv(SENTENCES_PATH)[["sentence"]]
keywords = pd.read_excel(KEYWORDS_PATH)

# check data
keywords["Category"].value_counts()

Category
SERIES      408
TYPE        273
BRAND        92
MATERIAL     42
NICKNAME     42
LINES        19
SOCIAL        3
Name: count, dtype: int64

In [2]:
keywords.loc[keywords["Category"] == "SOCIAL"]

,Category,stdkw
603,SOCIAL,保值
604,SOCIAL,爆款
605,SOCIAL,流行款


In [3]:
def sample_sentence(data: pd.DataFrame, n: int) -> list[dict]:
    """
    Sample n records from sentence data and return as a list of dictionaries.
    :param data: a dataframe
    :param n: the number of returned samples
    :return: a list of dictionaries
    """
    return data.sample(n).to_dict(orient="records")


def sample_keyword(data: pd.DataFrame, n: int, stratify: bool) -> list[dict]:
    """
    Sample n records from keywords data and return as a list of dictionaries.
    :param data: keywords dataframe
    :param n: number of samples if stratify is False, number of samples per category if stratify is True (except SOCIAL category, which is returned as a whole)
    :param stratify: indicate whether to stratify the samples by category
    :return: a list of dictionaries 
    """
    if not stratify:
        return data.sample(n).to_dict(orient="records")
    else:
        # stratify by category
        # store the result
        # handle the case where the category is SOCIAL
        r = [*data.loc[data["Category"] == "SOCIAL"].to_dict(orient="records")]

        # other categories
        stratified_samples = data[data["Category"] != "SOCIAL"].groupby("Category", group_keys=True).apply(
            lambda x: x.sample(n), include_groups=False)
        stratified_samples.index = stratified_samples.index.droplevel(1)
        stratified_samples.reset_index(drop=False, inplace=True)
        r.extend(stratified_samples.to_dict(orient="records"))

        return r

In [4]:
# examples
sample_sentence(sentences, 3)

[{'sentence': '不过买过nano speedy有整理需求，喜欢老花，可推荐皮质包包。'},
 {'sentence': '新疆人喜欢皮质款，喜欢黑色包，喜欢大包。'},
 {'sentence': '喜欢黑色小包，喜欢逸时这瓶香水。'}]

In [5]:
sample_keyword(keywords, 1, stratify=True)

[{'Category': 'SOCIAL', 'stdkw': '保值'},
 {'Category': 'SOCIAL', 'stdkw': '爆款'},
 {'Category': 'SOCIAL', 'stdkw': '流行款'},
 {'Category': 'BRAND', 'stdkw': 'off white'},
 {'Category': 'LINES', 'stdkw': '荔枝纹'},
 {'Category': 'MATERIAL', 'stdkw': '布艺'},
 {'Category': 'NICKNAME', 'stdkw': '月光女神'},
 {'Category': 'SERIES', 'stdkw': 'sunny'},
 {'Category': 'TYPE', 'stdkw': '裙装'}]

In [6]:
# connect to DeepSeek model
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(
    model='deepseek-chat',
    openai_api_key='sk-e9a452d4146a4d7286504bba5e8d7cc9',
    openai_api_base='https://api.deepseek.com',
    max_tokens=1024
)

parser = StrOutputParser()

In [7]:
def list2prompt(l: list[dict], data_type: str, category: str = "") -> str:
    """
    Convert a list of dictionaries to a string prompt.
    :param l: a list of dictionaries
    :param data_type: can be either "sentence" or "keyword"
    :param category: used when data_type is "keyword"
    :return: a string prompt
    """
    if data_type == "sentence":
        return "\n".join(d["sentence"] for d in l)
    elif data_type == "keyword":
        return ",".join(d["stdkw"] for d in l if d["Category"] == category.upper())
    else:
        raise NotImplementedError(f"Data type {data_type} is not supported.")

In [8]:
# define prompt template
system_prompt = """你是一位经过专业培训的奢侈品柜台导购。现在我有一些用于培训的示范用语，这些语言主要是简要地对客户进行描述。请参考以下这些语言示范，并考虑现有的产品信息，深呼吸，思考一下，你能举出其他的例子吗？这些例子应该尽可能贴近原有示范的风格，产品相关信息请严格按照我们提供的产品列表。"""

user_prompt = """以下是供你参考的产品信息（属性）列表：

社交属性：{SOCIAL}
品牌：{BRAND}
纹路：{LINES}
材料：{MATERIAL}
昵称：{NICKNAME}
系列：{SERIES}
类型：{TYPE}

请继续根据这些示例，生成{n_sentences}个新的句子。以下是一些示例：
{SENTENCE}
"""

# user_prompt = """以下是供你参考的产品信息（属性）列表：
# 
# 社交属性：{SOCIAL}
# 品牌：{BRAND}
# 纹路：{LINES}
# 材料：{MATERIAL}
# 昵称：{NICKNAME}
# 系列：{SERIES}
# 类型：{TYPE}
# 
# 请帮助我模仿下面的句子的语言风格，替换掉相关的产品属性。句子长度没有固定要求，但要保持自然流畅，可以仅使用产品的部分属性。以下是一些示例：
# {SENTENCE}
# 请继续根据这些示例，生成{n_sentences}个新的句子。
# """

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", user_prompt)
    ]
)

# a = sample_sentence(sentences, 4)
# b = sample_keyword(keywords, 2, stratify=True)

# input_prompt = prompt_template.invoke({
#     "SOCIAL": list2prompt(b, "keyword", "SOCIAL"),
#     "BRAND": list2prompt(b, "keyword", "BRAND"),
#     "LINES": list2prompt(b, "keyword", "LINES"),
#     "MATERIAL": list2prompt(b, "keyword", "MATERIAL"),
#     "NICKNAME": list2prompt(b, "keyword", "NICKNAME"),
#     "SERIES": list2prompt(b, "keyword", "SERIES"),
#     "TYPE": list2prompt(b, "keyword", "TYPE"),
#     "SENTENCE": list2prompt(sample_sentence(sentences, 3), "sentence"),
#     "n_sentences": "1"
# })

In [11]:
def chain_pipeline(llm_prompt_template, llm_model, llm_parser, debug: bool = False) -> str:
    """
    Use langchain to build a chain of models.
    :param llm_prompt_template: input prompt
    :param llm_model: model for generating text
    :param llm_parser: parser for parsing the output
    :param debug: print debug information
    :return: model output in string format
    """
    # generate samples as model prompt
    s = sample_sentence(sentences, 4)
    k = sample_keyword(keywords, 8, stratify=True)
    
    if debug:
        print(f"example sentences: {s}")
        print(f"example keywords: {k}")
        
    chain = llm_prompt_template | llm_model | llm_parser
    r = chain.invoke({
        "SOCIAL": list2prompt(k, "keyword", "SOCIAL"),
        "BRAND": list2prompt(k, "keyword", "BRAND"),
        "LINES": list2prompt(k, "keyword", "LINES"),
        "MATERIAL": list2prompt(k, "keyword", "MATERIAL"),
        "NICKNAME": list2prompt(k, "keyword", "NICKNAME"),
        "SERIES": list2prompt(k, "keyword", "SERIES"),
        "TYPE": list2prompt(k, "keyword", "TYPE"),
        "SENTENCE": list2prompt(s, "sentence"),
        "n_sentences": "1"
    })
    return r

In [13]:
chain_pipeline(prompt_template, model, parser, debug=True)

example sentences: [{'sentence': '年轻女性，喜欢歌曲，喜欢迪奥，爱好美食，买过经典款女包，法棍到货通知。'}, {'sentence': '不过买过nano speedy有整理需求，喜欢老花，可推荐皮质包包。'}, {'sentence': '购买过petite malle，购买过never full中号。'}, {'sentence': '年轻女性，爱好旅游，爱好瑜伽，喜欢channel，喜欢celine。'}]
example keywords: [{'Category': 'SOCIAL', 'stdkw': '保值'}, {'Category': 'SOCIAL', 'stdkw': '爆款'}, {'Category': 'SOCIAL', 'stdkw': '流行款'}, {'Category': 'BRAND', 'stdkw': 'sandro'}, {'Category': 'BRAND', 'stdkw': '登喜路'}, {'Category': 'BRAND', 'stdkw': '杜嘉班纳'}, {'Category': 'BRAND', 'stdkw': 'loewe'}, {'Category': 'BRAND', 'stdkw': '杰尼亚'}, {'Category': 'BRAND', 'stdkw': 'chaumet'}, {'Category': 'BRAND', 'stdkw': 'ysl'}, {'Category': 'BRAND', 'stdkw': 'roger vivier'}, {'Category': 'LINES', 'stdkw': '夜光'}, {'Category': 'LINES', 'stdkw': '荔枝纹'}, {'Category': 'LINES', 'stdkw': '压纹'}, {'Category': 'LINES', 'stdkw': '灰花'}, {'Category': 'LINES', 'stdkw': '波点'}, {'Category': 'LINES', 'stdkw': '马赛克'}, {'Category': 'LINES', 'stdkw': 'monogram'}, {'Category': 'LINES', 'stdkw': 

'中年男士，热衷于高尔夫，偏爱登喜路，对时尚有着独到见解，曾购入杰尼亚的正装系列，对保值的鳄鱼皮皮带有浓厚兴趣，希望得到新品到货的通知。'

In [ ]:
# deprecated code snippet
# a = keywords.groupby("Category", group_keys=True).apply(lambda x: x.sample(2), include_groups=False) # .to_dict(orient="records")
# a.index = a.index.droplevel(1)
# a.reset_index(drop=False, inplace=True)
# a.to_dict(orient="records")